# LOADING META-DATA IN CSV FILE

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/05-loading-data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Start EVA Server
We are reusing the start server notebook for launching the EVA server

In [1]:
!wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

File ‘00-start-eva-server.ipynb’ already there; not retrieving.



nohup eva_server >> eva.log 2>&1 &
('YOLOv5 🚀 2023-4-12 Python-3.9.2 torch-2.0.0+cu117 CPU\n', '\n', 'Downloading https://github.com/ultralytics/yolov5/releases/download/v7.0/yolov5s.pt to yolov5s.pt...\n')
('\n', '  0%|          | 0.00/14.1M [00:00<?, ?B/s]\n', ' 10%|█         | 1.48M/14.1M [00:00<00:00, 15.5MB/s]\n')
(' 62%|██████▏   | 8.70M/14.1M [00:00<00:00, 50.8MB/s]\n', '100%|██████████| 14.1M/14.1M [00:00<00:00, 55.6MB/s]\n', '\n')
('Fusing layers... \n', 'YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients\n', 'Adding AutoShape... \n')
('04-12-2023 03:02:25 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF FastRCNNObjectDetector already exists, nothing added.\n', '04-12-2023 03:02:25 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF ArrayCount already exists, nothing added.\n', '04-12-2023 03:02:25 WARNING[create_udf_executor:create_udf_executor.py:exec:0101] UDF Crop already exists, nothing added.\n')
('04-12-2023 03:02:25 WARNING[crea

Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.insert(0,'..')
from eva.server.db_api import connect

import nest_asyncio
nest_asyncio.apply()
connection = connect(host = '0.0.0.0', port = 5432) # hostname, port of the server where EVADB is running

cursor = connection.cursor()

### Video Files
 - Download a subset of the Berkeley Deepdrive dataset from Dropbox

In [3]:
# sample dataset of 2 videos
!wget -nc https://www.dropbox.com/s/kg1q69ijbhjfecw/bddtest.zip

# unzip
!unzip -n bddtest.zip

File ‘bddtest.zip’ already there; not retrieving.



/bin/bash: line 1: unzip: command not found


## Load 2 BDD videos

In [4]:
cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a2e3ca5c856cde.mp4" INTO bddtest_1;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD VIDEO "./bddtest/videos/bddtest_00a395fed60c0b47.mp4" INTO bddtest_2;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: Load VIDEO failed: encountered unexpected error Load VIDEO failed due to no valid files found on path bddtest/videos/bddtest_00a2e3ca5c856cde.mp4
@status: ResponseStatus.FAIL
@batch: 
 None
@error: Load VIDEO failed: encountered unexpected error Load VIDEO failed due to no valid files found on path bddtest/videos/bddtest_00a395fed60c0b47.mp4


## Create a BDD Meta-data Table

In [5]:
create_table_query = f""" 

    CREATE TABLE IF NOT EXISTS bddtestmeta(
        id INTEGER UNIQUE,
        frame_id INTEGER,
        video_id INTEGER,
        dataset_name TEXT(30),
        label TEXT(30),
        bbox NDARRAY FLOAT32(4),
        object_id INTEGER
    );
    
    """

cursor.execute(create_table_query)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.10775238299993362


## Load the 2 corresponding CSV files associated with the 2 previously loaded videos

In [6]:
cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a2e3ca5c856cde.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

cursor.execute("LOAD CSV 'bddtest/info/bddtest_00a395fed60c0b47.csv' INTO bddtestmeta;")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: The dataset file could not be found. Please verify that the file exists in the specified path.
@status: ResponseStatus.FAIL
@batch: 
 None
@error: The dataset file could not be found. Please verify that the file exists in the specified path.


## Run a Query over the Video dataset


In [7]:
cursor.execute("""SELECT id, FastRCNNObjectDetector(data) 
                  FROM bddtest_1 
                  WHERE id < 20;""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.FAIL
@batch: 
 None
@error: bddtest_1 does not exist. Create the table using CREATE TABLE.


## Run a query over the Meta-data Table

In [8]:
cursor.execute("""SELECT *
                  FROM bddtestmeta
                  WHERE bddtestmeta.video_id = 4;""")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.02111088599986033
